### Necessary imports

In [1]:
!pip install -q -U torch datasets transformers tensorflow langchain playwright html2text sentence_transformers faiss-cpu
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.4/37.4 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2

### Dependencies

In [2]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

### Load quantized Mistal 7B

In [3]:
#################################################################
# Tokenizer
#################################################################

model_name='mistralai/Mistral-7B-Instruct-v0.1'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

### Count number of trainable parameters

In [4]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 262410240
all model parameters: 3752071168
percentage of trainable model parameters: 6.99%


### Build Mistral text generation pipeline

In [5]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

In [6]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

### Load and chunk documents. Load chunked documents into FAISS index

In [ ]:
!playwright install
!playwright install-deps

In [8]:
import nest_asyncio
nest_asyncio.apply()

# Articles to index
nfl_articles = ["https://www.fantasypros.com/2023/11/rival-fantasy-nfl-week-10/",
            "https://www.fantasypros.com/2023/11/5-stats-to-know-before-setting-your-fantasy-lineup-week-10/",
            "https://www.fantasypros.com/2023/11/nfl-week-10-sleeper-picks-player-predictions-2023/",
            "https://www.fantasypros.com/2023/11/nfl-dfs-week-10-stacking-advice-picks-2023-fantasy-football/",
            "https://www.fantasypros.com/2023/11/players-to-buy-low-sell-high-trade-advice-2023-fantasy-football/"]

nba_articles =["https://www.fantasypros.com/2024/01/fantasy-basketball-trade-advice-miles-bridges-dejounte-murray-jaren-jackson-jr-cole-anthony/",
               "https://www.fantasypros.com/2024/01/fantasy-basketball-trade-advice-scottie-barnes-myles-turner-vince-williams-jr-grayson-allen/",
               "https://www.fantasypros.com/2024/01/mondays-nba-sleeper-picks-player-predictions-kyrie-irving-klay-thompson-immanuel-quickley-1-15/",
               "https://www.fantasypros.com/2024/01/fantasy-basketball-category-analysis-waiver-wire-pickups-week-13-2024/",
               "https://www.fantasypros.com/2024/01/mondays-nba-sleeper-picks-player-predictions-jaren-jackson-jr-jalen-duren-jaden-ivey-1-22/"]

# Scrapes the blogs above
loader = AsyncChromiumLoader(nba_articles)
docs = loader.load()

In [9]:
# Converts HTML to plain text
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

# Chunk text
text_splitter = CharacterTextSplitter(chunk_size=100,
                                      chunk_overlap=0)
chunked_documents = text_splitter.split_documents(docs_transformed)

# Load chunked documents into the FAISS index
db = FAISS.from_documents(chunked_documents,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

retriever = db.as_retriever()

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


### Create PromptTemplate and LLMChain

In [10]:
prompt_template = """
### [INST] Instruction: Answer the question based on your fantasy basketball knowledge. Here is context to help:

{context}

### QUESTION:
{question} [/INST]
 """

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

Previous answer: '{\n  "probability": 0.45,\n  "confidence_interval": [0.4, 0.5]\n}'

Without probability

In [12]:
llm_chain.invoke({"context": "", "question": "In the NBA basketball game, scheduled for October 27th in 2023, which team is going to win: Pistons or Hornets?"})

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'context': '',
 'question': 'In the NBA basketball game, scheduled for October 27th in 2023, which team is going to win: Pistons or Hornets?',
 'text': "\nBased on my fantasy basketball knowledge, I would predict that the Charlotte Hornets will win the game against the Detroit Pistons on October 27th in 2023. The Hornets have a strong roster with players such as LaMelo Ball, Gordon Hayward, and Miles Bridges, who can score points and control the game. On the other hand, the Pistons have a young and rebuilding team with players such as Cade Cunningham and Saddiq Bey, who may struggle to keep up with the experienced and talented Hornets. However, it's important to note that this is just a prediction based on my knowledge and there are many factors that can affect the outcome of a basketball game."}

With probability

In [13]:
prompt_template = """
### [INST] Instruction: Answer the question based on your fantasy basketball knowledge. Here is context to help:

{context}
and add the probability and confidence interval of the answer
### QUESTION:
{question} [/INST]
 """

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain_prob = LLMChain(llm=mistral_llm, prompt=prompt)

In [14]:
llm_chain_prob.invoke({"context": "", "question": "In the NBA basketball game, scheduled for October 27th in 2023, which team is going to win: Pistons or Hornets?"})

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'context': '',
 'question': 'In the NBA basketball game, scheduled for October 27th in 2023, which team is going to win: Pistons or Hornets?',
 'text': "\nBased on my fantasy basketball knowledge, I predict that the Charlotte Hornets will win the game against the Detroit Pistons on October 27th in 2023. The probability of this happening is 65%, with a 95% confidence interval.\n\nThe Charlotte Hornets have a strong roster with players such as LaMelo Ball, Gordon Hayward, and Miles Bridges, who can score points and control the pace of the game. On the other hand, the Detroit Pistons have a young and inexperienced team, with players such as Cade Cunningham and Saddiq Bey, who may struggle to keep up with the Hornets' offense. Additionally, the Hornets have won their last three games against the Pistons, indicating a recent trend in favor of the Hornets.\n\nHowever, it's important to note that basketball games are unpredictable and can be influenced by various factors such as injuries, pl

So the model guessed the right answer and it explained perfectly the response based on relevant information provided within the articles.

### Build RAG Chain

In [15]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

result = rag_chain.invoke("Should I bet for Hornets in their next match?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [16]:
result['context']

[Document(page_content='The Hornets have four games this upcoming week. Charlotte, ranked 26th in the\nleague for three-pointers made, faces varied challenges in their upcoming\ngames. They could struggle against the Timberwolves (fourth in OPP 3PM),\nPistons (eighth in OPP 3PM) and Rockets (10th in OPP 3PM), who excel in three-\npoint defense, posing a significant hurdle given the Hornets’ existing\ndifficulties in this area. However, the matchup against the Jazz (27th in OPP\n3PM) appears more promising, with the Jazz’s lower defensive efficacy in\nthree-pointers providing an opportunity for the Hornets to improve their\nperformance.', metadata={'source': 'https://www.fantasypros.com/2024/01/fantasy-basketball-category-analysis-waiver-wire-pickups-week-13-2024/'}),
 Document(page_content='For the Hornets and Miller, the offensive strategy should be tailored to\nexploit the defensive gaps of the Pistons, Rockets and Jazz, where they have\nan edge due to the opponents’ higher defensive

In [17]:
print(result['text'])


Based on the information provided, it seems that the Hornets may face some challenges in their upcoming games, particularly against teams like the Timberwolves, Pistons, and Rockets, who excel in three-point defense. However, the matchup against the Jazz appears more promising, as the Jazz's lower defensive efficacy in three-pointers provides an opportunity for the Hornets to improve their performance.

It is also important to note that the offensive strategy for the Hornets and Miller should be tailored to exploit the defensive gaps of their opponents, where they have an edge due to the opponents' higher defensive ratings. Adapting their three-point shooting approach to each opponent's defensive characteristics and the pace of each game will be crucial for their success.

Overall, while there are potential challenges ahead for the Hornets, their offensive strategy and ability to adapt to their opponents' defensive characteristics could make them a strong contender in their upcoming g